In [ ]:
import scipy.io as scio
import numpy as np
import tensorflow as tf
import keras
from keras.layers import Input, Dense, ZeroPadding2D, Dropout, Activation
from keras.layers import Input, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras import layers
import time
import warnings
import os
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.initializers import glorot_uniform
from keras.applications.imagenet_utils import preprocess_input
from keras.utils import layer_utils
from keras.utils.vis_utils import plot_model
from keras.utils.data_utils import get_file
from keras.utils.vis_utils import model_to_dot

def EMGNet_block(inpt, filters=64, kernel_size=(3, 3), strides=1, padding='same'):
    x = layers.Conv2D(filters=filters, kernel_size=(6, 2), strides=strides, padding='valid')(inpt)
    x = layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x_shortcut = x
    x = conv2d_bn(x, filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)
    x = conv2d_bn(x, filters=filters, kernel_size=kernel_size)
    x = layers.add([x, x_shortcut])
    x_shortcut = x
    x = conv2d_bn(x, filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)
    x = conv2d_bn(x, filters=filters, kernel_size=kernel_size)
    x = layers.add([x, x_shortcut])
    return x

def conv2d_bn(inpt, filters=64, kernel_size=(3, 3), strides=1, padding='same'):
    x = layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)(inpt)
    x = layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    return x


def basic_bottle(inpt, filters=64, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False):
    x = conv2d_bn(inpt, filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)
    x = conv2d_bn(x, filters=filters)
#     x = layers.Dropout(0.1)
    x = layers.BatchNormalization()(x)
    if if_baisc == True:
        temp = conv2d_bn(inpt, filters=filters, kernel_size=(1, 1), strides=2, padding='same')
        outt = layers.add([x, temp])
    else:
        outt = layers.add([x, inpt])
    return outt

def identity_block(X, f, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base   = 'bn'  + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    # save the input value 
    X_shortcut = X

    # first component of main path
    X = Conv2D(filters=F1, kernel_size=(1,1), strides=(1,1), padding='valid',
               name=conv_name_base+'2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+'2a')(X)
    X = Activation('relu')(X)

    # second component of main path
    X = Conv2D(filters=F2, kernel_size=(f,f), strides=(1,1), padding='same',
               name=conv_name_base+'2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+'2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1,1), strides=(1,1), padding='valid',
               name=conv_name_base+'2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+'2c')(X)

    # Final step
    # Add shortcut value to main path, and pass it through a ReLU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X
def convolutional_block(X, f, filters, stage, block, s=2):
    """
    Implementation of the identity block 

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, 主路径中间的那个CONV的窗口形状
    filters -- python整数列表, 定义主路径每个CONV层中的滤波器的数量
    stage --整数，用于命名层，取决于他们在网络中的位置          阶段
    block --字符串/字符，用于命名层，取决于他们在网络中的位置   块
    s -- 整数，指定滑动的大小

    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base   = 'bn'  + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    # save the input value 
    X_shortcut = X

    # first component of main path
    X = Conv2D(filters=F1, kernel_size=(1,1), strides=(s,s), padding='valid',
               name=conv_name_base+'2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+'2a')(X)
    X = Activation('relu')(X)

    # second component of main path
    X = Conv2D(filters=F2, kernel_size=(f,f), strides=(1,1), padding='same',
               name=conv_name_base+'2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+'2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1,1), strides=(1,1), padding='valid',
               name=conv_name_base+'2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+'2c')(X)

    # shortcut path
    X_shortcut = Conv2D(filters=F3, kernel_size=(1,1), strides=(s,s), padding='valid',
            name=conv_name_base+'1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base+'1')(X_shortcut)

    # Final step
    # Add shortcut value to main path, and pass it through a ReLU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X


def ResNet50(input_shape, classes=10):

    X_input = Input(input_shape)

    # stage 1
    X = Conv2D(filters=4, kernel_size=(3,3), strides=(1,1), padding='valid',
               name='conv1', kernel_initializer=glorot_uniform(seed=0))(X_input)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2,2), strides=(2,2))(X)

    # stage 2
    X = convolutional_block(X, f=3, filters=[64,64,256], stage=2, block='a', s=1)
    X = identity_block(X, f=3, filters=[64,64,256], stage=2, block='b')
    X = identity_block(X, f=3, filters=[64,64,256], stage=2, block='c')

    # stage 3
    X = convolutional_block(X, f=3, filters=[128,128,512], stage=3, block='a', s=1)
    X = identity_block(X, f=3, filters=[128,128,512], stage=3, block='b')
    X = identity_block(X, f=3, filters=[128,128,512], stage=3, block='c')
    X = identity_block(X, f=3, filters=[128,128,512], stage=3, block='d')

    # stage 4
    X = convolutional_block(X, f=3, filters=[256,256,1024], stage=4, block='a', s=1)
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block='b')
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block='c')
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block='d')
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block='e')
    X = identity_block(X, f=3, filters=[256,256,1024], stage=4, block='f')

    # stage 5
    X = convolutional_block(X, f=3, filters=[256,256,1024], stage=5, block='a', s=1)
    X = identity_block(X, f=3, filters=[256,256,1024], stage=5, block='b')
    X = identity_block(X, f=3, filters=[256,256,1024], stage=5, block='c')

    # fully connected output layer
    X = Flatten(name='flatten')(X)
    X = Dense(512,    activation='relu',    name='fc1')(X)
    X = Dense(classes, activation='softmax', name='fc2')(X)

    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model


def ResNet18(input_shape, classes=10):
    X_input = Input(input_shape)

    X = conv2d_bn(X_input, filters=64, kernel_size=(4, 4), strides=1, padding='valid') # 7 7 64

    # layer 2
    X = basic_bottle(X, filters=64, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)#7 7 64
    X = basic_bottle(X, filters=64, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)#7 7 64
    X = layers.Dropout(.2)(X)
    # layer 3
    X = basic_bottle(X, filters=128, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True) #
    X = basic_bottle(X, filters=128, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    X = layers.Dropout(.2)(X)
    # layer 4
    X = basic_bottle(X, filters=256, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    X = basic_bottle(X, filters=256, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    X = layers.Dropout(.2)(X)
    # layer 5
    X = basic_bottle(X, filters=512, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    X = basic_bottle(X, filters=512, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    X = layers.Dropout(.2)(X)
    # GlobalAveragePool
    X = layers.GlobalAveragePooling2D()(X)
    X = layers.Dropout(.5)(X)
    X = layers.Dense(classes, activation='softmax')(X)

    model = Model(inputs=X_input, outputs=X, name='ResNet18')
    return model


def VGG16(input_shape, classes=10):

    X_input = Input(input_shape)

    "block 1"
    X = layers.Conv2D(filters=4, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='block1_conv1')(X_input)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Conv2D(filters=4, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='block1_conv2')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.AveragePooling2D((2,2), strides=(2,2), name='block1_pool')(X)
    X = layers.Dropout(.2)(X)


    "block 2"
    X = layers.Conv2D(filters=8, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='block2_conv1')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Conv2D(filters=8, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='block2_conv2')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.AveragePooling2D((2,2), strides=(2,2), name='block2_pool')(X)
#     X = layers.Dropout(.5)(X)

    "block 3"
    X = layers.Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='block3_conv1')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='block3_conv2')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='block3_conv3')(X)
    X = layers.BatchNormalization(axis=3)(X)
#     X = layers.Dropout(.5)(X)
#     X = layers.AveragePooling2D((2,2), strides=(2,2), name='block3_pool')(X)

    "block 4"
    X = layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='block4_conv1')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='block4_conv2')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='block4_conv3')(X)
    X = layers.BatchNormalization(axis=3)(X)
#     X = layers.Dropout(.5)(X)
#     X = layers.AveragePooling2D((2,2), strides=(2,2), name='block4_pool')(X)

    "block 5"
    X = layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='block5_conv1')(X)
    X = layers.BatchNormalization(axis=3)(X)    
    X = layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='block5_conv2')(X)
    X = layers.BatchNormalization(axis=3)(X)
    X = layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same', name='block5_conv3')(X)
    X = layers.BatchNormalization(axis=3)(X)
#     X = layers.Dropout(.5)(X)
#     X = layers.AveragePooling2D((2,2), strides=(2,2), name='block5_pool')(X)
# 
    X = layers.Flatten(name='flatten')(X)
    X = layers.Dropout(.5)(X)
    X = layers.Dense(256,    activation='relu',    name='fc1')(X)
    X = layers.Dropout(.5)(X)
    X = layers.Dense(classes, activation='softmax', name='fc2')(X)

    model = Model(inputs=X_input, outputs=X, name='VGG16')

    return model


def ANN(input_shape, classes=10):

    model = keras.Sequential([
        keras.layers.Flatten(input_shape=input_shape),
#         keras.layers.Dropout(0.5),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(512, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(classes, activation='softmax')                            
    ])

    return model


def CNN(input_shape, classes=10):

    X_input = Input(input_shape)

    "block 1"
    "32 filters,  a row of the length of number of electrodes,  ReLU"
    X = layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1),padding='same', name='conv1')(X_input)
    X = layers.BatchNormalization()(X)
    X = layers.Activation('relu', name='relu1')(X)
    X = layers.Dropout(.2)(X)
    X = layers.AveragePooling2D((2,2), strides=(2,2), name='pool1')(X)
    
#     "block 3"
#     "64 filters 5*5,  ReLu,  average pool 3*3"
    X = layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1),padding='same', name='conv3')(X)
    X = layers.BatchNormalization()(X)
    X = layers.Activation('relu', name='relu3')(X)
    X = layers.MaxPooling2D((2,2), strides=(1,1), name='pool2')(X)
    X = layers.Dropout(.5)(X)

#     "block 4"
#     "64 filters 5*1,  ReLU"
#     X = layers.Conv2D(filters=64, kernel_size=(5,1), strides=(1,1),padding='same', name='conv4')(X)
# #     X = layers.BatchNormalization()(X)
#     X = layers.Activation('relu', name='relu4')(X)

    "block 5"
    "filters 1*1,  softmax loss"
#     X = layers.Conv2D(filters=32, kernel_size=(1,1), strides=(1,1),padding='same', name='conv5')(X)
#     X = layers.BatchNormalization()(X)
    X = layers.Flatten(name='flatten')(X)
    X = layers.Dropout(.5)(X)
    X = layers.Dense(256,    activation='relu',    name='fc1')(X)
    X = layers.Dropout(.5)(X)
    X = layers.Dense(classes, activation='softmax', name='fc2')(X)

    model = Model(inputs=X_input, outputs=X, name='CNN')
    return model

    return model


def EMGNet(input_shape, classes=10):
    X_input = Input(input_shape)

    X = EMGNet_block(X_input, filters=64, kernel_size=(4, 2), strides=1, padding='same')
    X = EMGNet_block(X, filters=128, kernel_size=(4, 2), strides=1, padding='same')
    X = EMGNet_block(X, filters=256, kernel_size=(4, 2), strides=1, padding='same')
    X = EMGNet_block(X, filters=512, kernel_size=(4, 2), strides=1, padding='same')
    X = layers.GlobalAveragePooling2D()(X)
    X = layers.Dropout(.5)(X)
    X = layers.Dense(classes, activation='softmax')(X)

    model = Model(inputs=X_input, outputs=X, name='EMGNet')
    return model